In [ ]:
!pip install gekko

In [ ]:
from gekko import GEKKO
import numpy as np
import random
from contextlib import contextmanager
import sys
import os
from rep_code import *
import matplotlib.pyplot as plt

Rep Code Decoder

In [ ]:
# rep code decoder
@contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout  # Backup current stdout
        sys.stdout = devnull  # Redirect stdout to devnull (suppress)
        try:
            yield
        finally:
            sys.stdout = old_stdout  # Restore original stdout
# settings ----
num_qubits = 3
num_stabilizers = num_qubits - 1
p = 0.1

ps = {} # error probs
for i in range(num_qubits):
    ps[i] = p # constant prob of error for now

Cs = { # stabilizer check values, these are the measured stabilizer values
    (0,1): -1, # Parity check value: -1 
    (1,2): 1 # Parity check value: 1
}
# The above stabilizers indicate Z0Z1 = -1 and Z1Z2 = 1, so there's probably an error on the 0th qubit
# Let's see if the MLE decoder can figure this out

# set up gekko ----
m = GEKKO(remote=False)
m.options.SOLVER = 1  # APOPT is an MINLP solver
# optional solver settings with APOPT
m.solver_options = [
    "minlp_maximum_iterations 500",  # minlp iterations with integer solution
    "minlp_max_iter_with_int_sol 10",  # treat minlp as nlp
    "minlp_as_nlp 0",  # nlp sub-problem max iterations
    "nlp_maximum_iterations 50",  # 1 = depth first, 2 = breadth first
    "minlp_branch_method 1",  # maximum deviation from whole number
    "minlp_integer_tol 0.05",  # covergence tolerance
    "minlp_gap_tol 0.01",
    "print_level 0",
]


# set up variables ---
Es = {}
for i in range(num_qubits):
    Es[i] = m.Var(
        value=random.randint(0,1), lb=0, ub=1, integer=True
    )

Ks = {}
for k in Cs.keys():
    Ks[k] = m.Var(
        value=random.randint(0,1), lb=0, integer=True
    )


# Objective ---
m.Obj(
    -m.sum(
        [
            np.log(ps[j]) * Es[j]  + np.log(1 - ps[j]) * (1 - Es[j])
            for j in range(num_qubits)
        ]
    )
)


# Constraints ---

for key, val in Cs.items():
    i, j = key
    m.Equation(Es[i] + Es[j] - 2 * Ks[key] == (1 - val)/2)

with suppress_stdout():
    m.solve()

print("Solution (0: no error, 1: error): ", Es)
Es[0][0] == 1

In [ ]:
physical_error_probs = np.linspace(0.01, 1.0, num=10)#physical error probs
physical_error_probs

Graph

In [ ]:
# rep code decoder
@contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout  # Backup current stdout
        sys.stdout = devnull  # Redirect stdout to devnull (suppress)
        try:
            yield
        finally:
            sys.stdout = old_stdout  # Restore original stdout
# settings ----
num_qubits = 3
num_stabilizers = num_qubits - 1
# p = 0.1

physical_error_probs = np.linspace(0.01, 0.99, num=50)#physical error probs
logical_error_probs =[]
for p in physical_error_probs: 
    count = 0 #number of correct guesses from solver
    for i in range(1024):
        ps = {i: p for i in range(num_qubits)}
        error_string = create_error(num_qubits, p)

        Cs = construct_stabilizers(num_qubits, error_string)


        # set up gekko ----
        m = GEKKO(remote=False)
        m.options.SOLVER = 1  # APOPT is an MINLP solver
            # optional solver settings with APOPT

    
        m.solver_options = [
            "minlp_maximum_iterations 500",  # minlp iterations with integer solution
            "minlp_max_iter_with_int_sol 10",  # treat minlp as nlp
            "minlp_as_nlp 0",  # nlp sub-problem max iterations
            "nlp_maximum_iterations 50",  # 1 = depth first, 2 = breadth first
            "minlp_branch_method 1",  # maximum deviation from whole number
            "minlp_integer_tol 0.05",  # covergence tolerance
            "minlp_gap_tol 0.01",
            "print_level 0",
        ]
        

        # set up variables ---
        Es = {}
        for i in range(num_qubits):
            Es[i] = m.Var(
                value=random.randint(0,1), lb=0, ub=1, integer=True
            )

        Ks = {}
        for k in Cs.keys():
            Ks[k] = m.Var(
                value=random.randint(0,1), lb=0, integer=True
            )


        # Objective ---
        m.Obj(
            -m.sum(
                [
                    np.log(ps[j]) * Es[j]  + np.log(1 - ps[j]) * (1 - Es[j])
                    for j in range(num_qubits)
                ]
            )
        )


        # Constraints ---

        for key, val in Cs.items():
            i, j = key
            m.Equation(Es[i] + Es[j] - 2 * Ks[key] == (1 - val)/2)

        with suppress_stdout():
            m.solve()
        
        if(solver_accuracy(num_qubits, error_string, Es)!=True):
            count+=1
    logical_error_probs.append(count/1024)

print(logical_error_probs)

In [ ]:
physical_error_probs = np.linspace(0.01, 0.99, num=50)
physical_error_probs

In [ ]:
def plot(physical, logical):
    fig,ax = plt.subplots(1,1,dpi=200,figsize = (4,3))
    ax.plot(physical,logical)
    ax.legend(fontsize=6)
    ax.set_xlabel("Physical Error Rate (p)")
    ax.set_ylabel("Logical Error Rate")
    plt.plot()
plot(physical_error_probs, logical_error_probs)

In [ ]:
# rep code decoder
@contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout  # Backup current stdout
        sys.stdout = devnull  # Redirect stdout to devnull (suppress)
        try:
            yield
        finally:
            sys.stdout = old_stdout  # Restore original stdout
# settings ----
num_qubits = 5
num_stabilizers = num_qubits - 1
# p = 0.1

physical_error_probs = np.linspace(0.01, 0.99, num=50)#physical error probs
logical_error_probs =[]
for p in physical_error_probs: 
    count = 0 #number of correct guesses from solver
    for i in range(1024):
        ps = {i: p for i in range(num_qubits)}
        error_string = create_error(num_qubits, p)

        Cs = construct_stabilizers(num_qubits, error_string)


        # set up gekko ----
        m = GEKKO(remote=False)
        m.options.SOLVER = 1  # APOPT is an MINLP solver
            # optional solver settings with APOPT

    
        m.solver_options = [
            "minlp_maximum_iterations 500",  # minlp iterations with integer solution
            "minlp_max_iter_with_int_sol 10",  # treat minlp as nlp
            "minlp_as_nlp 0",  # nlp sub-problem max iterations
            "nlp_maximum_iterations 50",  # 1 = depth first, 2 = breadth first
            "minlp_branch_method 1",  # maximum deviation from whole number
            "minlp_integer_tol 0.05",  # covergence tolerance
            "minlp_gap_tol 0.01",
            "print_level 0",
        ]
        

        # set up variables ---
        Es = {}
        for i in range(num_qubits):
            Es[i] = m.Var(
                value=random.randint(0,1), lb=0, ub=1, integer=True
            )

        Ks = {}
        for k in Cs.keys():
            Ks[k] = m.Var(
                value=random.randint(0,1), lb=0, integer=True
            )


        # Objective ---
        m.Obj(
            -m.sum(
                [
                    np.log(ps[j]) * Es[j]  + np.log(1 - ps[j]) * (1 - Es[j])
                    for j in range(num_qubits)
                ]
            )
        )


        # Constraints ---

        for key, val in Cs.items():
            # i, j = key
            m.Equation(m.sum([Es[i] for i in key]) - 2 * Ks[key] == (1 - val)/2)

        with suppress_stdout():
            m.solve()
        
        if(solver_accuracy(num_qubits, error_string, Es)!=True):
            count+=1
    logical_error_probs.append(count/1024)

print(logical_error_probs)

In [ ]:
plot(physical_error_probs, logical_error_probs)